# *FEniCS* tutorial

# Import libraries

In [ ]:
from __future__ import print_function # Allow function print()
from IPython.display import clear_output

%matplotlib inline

In [ ]:
try:
    from fenics import *
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    from fenics import *

try:
    from fenics_adjoint import *
except ImportError:
    !pip install git+https://github.com/dolfin-adjoint/pyadjoint.git@2019.1.0
    from fenics_adjoint import *

try:
    from rbnics import *
except ImportError:
    !pip3 install git+https://github.com/RBniCS/RBniCS.git
    from rbnics import *
import rbnics.utils.config
assert "dolfin" in rbnics.utils.config.config.get("backends", "required backends")

# N.B. Errors like: a) 'dolfin.cpp.mesh.Mesh' object has no attribute '_ad_will_add_as_dependency'
#                   b) 'Function' object has no attribute 'block_variable'
# are due to conflicts between dolfin/fenics/dolfin_adjoint/fenics_adjoint.
# Possible solutions: 1) Try to import just one library
#                     2) Use namescapes (e.g. dolfin.solve instead of just solve)
#                     3) refine the mesh to overcome problem (a)
#                     4) Use different tools (e.g. RectangleMesh instead of mshr utilities to overcome (a))
#                     5) Use pause_annotation() and continue_annotation() to stop annotation by fenics_adjoint (this would overcome problem (b) since block variables are expolited for annotation)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define, evaluate and plot a function

In [ ]:
f = Constant(-6.0)

k = 1
f = Constant(k)

f = Expression('exp(-10.0 * (pow(x[0] - 0.75, 2) + pow(x[1] - 0.75, 2)))', degree = 2)

k = 1
f = Expression('4 * exp(-pow(k, 2) * (pow(x[0], 2) + pow(x[1] - 0.6, 2)))', degree = 1, k = k)

k = 1
f1 = Expression("sin(pi * x[0]) * sin(pi * x[1])", degree = 3)
f = Expression("f1 * k", f1 = f1, k = k, degree=3)

k = 1
f = Expression('1 + x[0] * x[0] + x[1] * x[1] + 0.5 * k', degree = 2, k = k)
f.k = 2 # Change parameter value

f = Expression(('1.0', '1.0'), degree = 1) # Vectorial function

In [ ]:
# Evaluate a function

f(0,0) # Evaluate f in (0,0)

import numpy as np
y = np.linspace(-1, 1, 101)
points = [(0, y_) for y_ in y]
f_line = np.array([f(point) for point in points]) # Evaluate f in a set of points

In [ ]:
# Plot a function: an interpolation or projection to the finite element space is needed

f = interpolate(f, V) # It gives the function in V with same nodal values as f 
f = project(f, V) # It gives the function in V closer to f wrt the L2 norm
plot(f)
plot(f, mode = ...) # mode = "color" to avoid level curves, mode = "glyphs" to plot a vector field, mode = "warp" for 3D plots 
plot(f,
     wireframe = True,              # Use wireframe rendering
     interactive = False,           # Do not hold plot on screen
     scalarbar = False,             # Hide the color mapping bar
     hardcopy_prefix = "myplot",    # Default plotfile name
     scale = 2.0,                   # Scale the warping/glyphs
     title = "Fancy plot"           # Set your own title
     )

# Plot a function with colorbar
import pylab as plt
fig = plot(f, mode='color')
plt.colorbar(fig) 
plt.show()

# Save a plot
fig = plot(f)
plt.colorbar(fig) 
plt.title("f")
frame = plt.gca()
frame.axes.get_xaxis().set_visible(False)
frame.axes.get_yaxis().set_visible(False)
plt.axis('off')
plt.savefig('f.png', transparent = True, bbox_inches='tight')
plt.savefig('f.pdf', transparent = True, bbox_inches='tight')

In [ ]:
# Extra utilities for functions

f.vector()
f.vector().norm('l2')
f.function_space()
f.assign(g) # Copy 'g' in 'f'
f.assign(g, annotate = True) # Copy 'g' in 'f' and remember the dependency between them
f = g.copy() # Deep copy
f = g.copy(deepcopy = False) # Shallow copy

In [1]:
# N.B. FEniCS cannot restrict functions to boundaries or to subdomains, hence they are defined over the entire domain

# Vectors and Matrices

In [ ]:
b = as_vector([1.0, 2.0])
K = as_matrix([[1, 2], [3, 4]])
I = Identity(2)

A.axpy(float(1.0), B, same_nonzero_pattern = False) # A = A + 1.0*B

# Variational Problem
The finite element variational problem in general reads:
$$
\text{Find } u \in V \text{ such that  } a(u,v) = L(v) \quad \forall v \in V
$$


## Create the mesh

In [ ]:
# Create a simple mesh

n = 32 # Mesh refinement
mesh = UnitSquareMesh(n, n) # Mesh in the unit square with vertices (0,0), (0,1), (1,0), (1,1)
mesh = UnitSquareMesh(MPI.COMM_WORLD, n, n) # Parallelization
mesh = UnitSquareMesh(MPI.COMM_SELF, n, n)  # Separate mesh for each core (useful to perform several tests)

# N.B. To run in parallel: mpiexec -n 4 python name.py

nx = ny = 32
mesh = RectangleMesh(Point(-2, -2), Point(2, 2), nx, ny)

nx = ny = nz = 10
mesh = BoxMesh(Point(0, 0, 0), Point(1, 1, 1), 10, 10, 10)

In [ ]:
# Plot a mesh

plot(mesh)

In [ ]:
# Refine the mesh

mesh = refine(mesh)

# Refine the mesh in a subregion
class Subdomain(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], 1.0) and (between(x[1], (0.5, 0.7))
subdomain = Subdomain() # Alternative way to def the subdomain: subdomain = CompiledSubDomain('std::abs(x[0]-0.5) < 0.25 && std::abs(x[1]-0.5) < 0.25')
domains = MeshFunction("bool", mesh, mesh.geometric_dimension())
domains.set_all(False)
subdomain.mark(domains, True)
mesh = refine(mesh, domains)

# Alternative way to refine the mesh in a subregion (more precise)
domains = MeshFunction("bool", mesh, mesh.geometric_dimension())
domains.set_all(False)
for cell in cells(mesh):
    p = cell.midpoint()
    if p.distance(Point(0.0, 0.0)) < 0.1:
        domains[cell] = True
mesh = refine(mesh, domains)                                   

In [ ]:
# Define and mark a subregions (if mesh refinement is needed, do the refinement before)

# Define and mark a subdomain (recommended to go from the biggest to the smallest)
class Subdomain(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], 1.0) and (between(x[1], (0.5, 0.7))
subdomain = Subdomain()
# Alternative way to def the subdomain
# subdomain = CompiledSubDomain('std::abs(x[0]-0.5) < 0.25 && std::abs(x[1]-0.5) < 0.25')

domains = MeshFunction("size_t", mesh, mesh.geometric_dimension())
domains.set_all(0)
subdomain.mark(domains, 1) # or directly Subdomain().mark(domains, 1)
dx = Measure("dx", domain = mesh, subdomain_data = domains) # This will allow to integrate over the subdomain with dx(1)                           
                                    
# Define and mark a portion of the boundary (recommended to go from the biggest to the smallest)
class Gamma(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and (x[0]-10)**2 + (x[1]-5)**2 < 3**2
gamma = Gamma()
# Alternative way to def the portion
# gamma  = CompiledSubDomain("near(x[0], 0.)")

boundaries = MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
boundaries.set_all(0)
gamma.mark(boundaries, 1)
ds = Measure("ds", domain = mesh, subdomain_data = boundaries) # This will allow to integrate over the portion of the boundary with ds(1)

In [ ]:
# Create a mesh combining boolean operators (CSG = Constructive Solid Geometry) from mshr
# and combining them with operators + (union), * (intersection) and - (set difference)

# If the mesh is simple, prefer the previous commands rather than mshr

from mshr import *

S0 = Rectangle(Point(0, 0), Point(1, 1))
S1 = Rectangle(Point(0.5, 0.5),Point(1.5, 1.5))
C0 = Circle(Point(0.5, 0.5), 0.25)
C1 = Circle(Point(1,1), 0.25)
domain = (S0 + S1) - (C0 + C1)

n = 20 # Mesh refinement
mesh = generate_mesh(domain, n)
plot(mesh)

In [ ]:
# Import a mesh in .xml format

mesh = Mesh("name.xml")

# Import a mesh generated with mshr

mesh_xdmf = XDMFFile(MPI.comm_world, "name.xdmf")
mesh = Mesh()
mesh_xdmf.read(mesh)

In [ ]:
# To deal with mesh that deforms over time, see: https://www.dolfin-adjoint.org/en/latest/documentation/tube-shape-derivative/tube-shape-derivative.html
#                                                https://www.dolfin-adjoint.org/en/latest/documentation/stokes-shape-opt/stokes_problem.html

## Define the finite element space

In [ ]:
# Finite element space for a scalar
V = FunctionSpace(mesh, "CG", 1) # Continuous Galerkin; 'CG' is equivalent to 'P'
W = FunctionSpace(mesh, "DG", 0) # Discontinuous Galerkin
V = u_old.function_space()       # Retrieve the space from another function

# Finite element space for a vector
V = VectorFunctionSpace(mesh, 'P', 1) # Compact writing (each component belongs to the same space)

P1 = FiniteElement('P', triangle, 1)
element = MixedElement([P1, P1, P1]) # Now different spaces can be selected for each component
V = FunctionSpace(mesh, element)

V.sub(0) # Access subdomain of V
V1, V2 = V.split() # Access subdomains of V
V1 = V1.collapse() # V1 seen as new space with a new as-hoc basis (not as subspace of V with basis linked with V's basis)

u = Function(V)
u[0] # Retrieve single components as Indexed objects
u.sub(0) # Retrieve single components as Functions ("deepcopy = True" may be required in input)
u_1, u_2, u_3 = split(u) # Retrieve the components as Indexed objects
u_1, u_2, u_3 = u.split() # Retrieve the components as Functions ("deepcopy = True" may be required in input)
V_1, V_2, V_3 = V.split()

In [2]:
# N.B. nodal values of a vector function (2 components for example) are saved in the following way:
# f.vector() = [f_0(x_0), f_1(x_0), ..., f_0(x_n), f_1(x_n)]

## Define the boundary condition

In [ ]:
# Dirichlet boundary condition on the entire domain's boundary

def boundary(x, on_boundary):
    return on_boundary
bc = DirichletBC(V, 0, boundary)

bc = DirichletBC(V, 0, "on_boundary") # Compact writing

In [ ]:
# Dirichlet boundary condition on a portion of the domain boudnary

def boundary(x, on_boundary):
    d0 = sqrt((x[0] - 0.5) ** 2 + (x[1] - 0.5) ** 2)
    d1 = sqrt((x[0] - 1.0) ** 2 + (x[1] - 1.0) ** 2)
    return on_boundary and (d0 < 0.3 or d1 < 0.3)

def boundary(x, on_boundary):
    return on_boundary and x[0] < tol

bc = DirichletBC(V, 0.0, boundary)

In [ ]:
# Impose different boundary conditions

inflow  = 'near(x[0], 0)'
outflow = 'near(x[0], 1)'
walls   = 'near(x[1], 0) || near(x[1], 1)'
cylinder = 'on_boundary && x[0]>0.1 && x[0]<0.3 && x[1]>0.1 && x[1]<0.3'

bcu_noslip  = DirichletBC(V, Constant((0, 0)), walls)
bcp_inflow  = DirichletBC(Q, Constant(8), inflow)
bcp_outflow = DirichletBC(Q, Constant(0), outflow)
bcu = [bcu_noslip]
bcp = [bcp_inflow, bcp_outflow]

In [ ]:
# Non-homogeneous Dirichlet boundary condition

u_D = Expression('1 + x[0]*x[0] + 2*x[1]*x[1]', degree=2)

bc = DirichletBC(V, u_D, "on_boundary")

In [ ]:
# Time-dependent Dirichlet boundary condition: the time t will be updated in the time-loop

u_D = Expression('1 + x[0] * x[0] + x[1] * x[1] + 0.5 * t', degree=2, t=0)

bc = DirichletBC(V, u_D, "on_boundary")

## Define and solve the variational problem



In [ ]:
# Define the variational problem

u = TrialFunction(V)

v = TestFunction(V)

f = Expression('exp(-10.0 * (pow(x[0] - 0.75, 2) + pow(x[1] - 0.75, 2)))', degree = 2)

n  = FacetNormal(mesh) # Normal vector

a = inner(grad(u), grad(v)) * dx
L = f * v * dx

F = inner(grad(u), grad(v)) * dx - f * v * dx
a, L = lhs(F), rhs(F)

def q(u):
    return 1 + u**2
F = q(u) * inner(grad(u), grad(v)) * dx - f * v * dx

# N.B. use 'ds' for integrals over boundaries
#      use 'dx(1)' or 'ds(1)' to integrate over the subdomain / boundary's portion with mark=1

# N.B. For vectors:  (grad(v))_ij = dv_i / dx_j
#                    (nabla_grad(v))_ij = dv_j / dx_i
#      For matrices: (div(v))_i = dv_ij / dx_j
#                    (nabla_div(v))_j = dv_ij / dx_i

In [ ]:
# Solve the variational problem

u = Function(V)

solve(a == L, u, bc)

plot(u)

In [ ]:
# Solve with different parameters
# method 1

u = Function(V)
solve(a == L, u, bc, solver_parameters = {"linear_solver": "gmres",
                                          "preconditioner": "ilu"})

# method 2

param = LinearVariationalSolver.default_parameters()
param.linear_solver = 'gmres' # or 'lu' for example
param.preconditioner = 'ilu'
param.krylov_solver.absolute_tolerance = 1E-5
param.krylov_solver.relative_tolerance = 1E-3
param.krylov_solver.maximum_iterations = 1000
u = Function(V)
solve(a == L, u, bc, solver_parameters = param)

# method 3

u = Function(V)
problem = LinearVariationalProblem(a, L, u, bc)
solver = LinearVariationalSolver(problem)
param.linear_solver = 'gmres' # or 'lu' for example
param.preconditioner = 'ilu'
param.krylov_solver.absolute_tolerance = 1E-5
param.krylov_solver.relative_tolerance = 1E-3
param.krylov_solver.maximum_iterations = 1000
solver.solve() # Solution available in 'u'

In [ ]:
# Solve assembling the linear system

A = assemble(a)
b = assemble(L)
bc.apply(A, b)

A, b = assemble_system(a, L, bcs) # Assemble with one command

solve(A, u.vector(), b, 'bicgstab', 'hypre_amg') # 'bicgstab' is the linear solver, 'hyper_amg' is the preconditioner

In [ ]:
# Solve assembling the linear system with different parameters

A, b = assemble_system(a, L, bc)

solver = KrylovSolver('gmres', 'ilu') # or 'LUSolver()' for example
solver.parameters.absolute_tolerance = 1E-5
solver.parameters.relative_tolerance = 1E-3
solver.parameters.maximum_iterations = 1000

u = Function(V)
solver.solve(A, u.vector(), b)

In [ ]:
# Time-dependent case

T = 2.0            # Final time
num_steps = 10     # Number of time steps
dt = T / num_steps # Time step size
t = 0

u = TrialFunction(V)
v = TestFunction(V)

f = Expression('1 + x[0] * x[0] + x[1] * x[1] + 0.5 * t', degree = 2, t = t)

u_n = interpolate(Constant(0.0), V) # Initial value

a = u * v * dx + dt * inner(grad(u), grad(v)) * dx
L = (u_n + dt * f) * v * dx

u = Function(V)

for n in range(num_steps):

    # Update current time
    t += dt
    
    # Update time-dependent expressions
    f.t = t

    # Compute solution
    solve(a == L, u, bc)

    # Plot solution
    plot(u)

    # Update previous solution
    u_n.assign(u)

## Postprocessing

In [ ]:
# Save solution
output_file = HDF5File(MPI.comm_world, "name.h5", "w") # There is also XDMFile() for files .xdmf
output_file.write(u, "solution")
output_file.close()

File('path/name.pvd') << u # Paraview Format

# Load solution
u_new = Function(V)
input_file = HDF5File(MPI.comm_world, "name.h5", "r")
input_file.read(u, "solution")
input_file.close()

In [ ]:
# Compute errors
# L2 error
error_L2 = errornorm(u_ex, u) # L2 error by default
error_L2 = errornorm(u_ex, u, 'L2')
error_H1 = errornorm(u_ex, u, norm_type = 'H10', degree_rise = 3) # H1 seminorm

# Compute maximum error at vertices
vertex_values_u_ex = u_ex.compute_vertex_values(mesh)
vertex_values_u = u.compute_vertex_values(mesh)
import numpy as np
error_max = np.max(np.abs(vertex_values_u_D - vertex_values_u))

# Print errors
print('error_L2  =', error_L2)
print('error_max =', error_max)

# PDE-constrained Optimisation
$$
\min_{u,m} J(u,m) \\
\text{subject to}\\ F(u,m) = 0 \\
l_b \leq m \leq u_b\\
g(m) \geq 0
$$
where $m$ contains the optimisation variables, $J$ is a real valued objective functional, $F(u,m)=0$ is the PDE with solution $u$. The bounds and inequality constraints can be used to restrict the feasible optimisation variables.

# Reduced problem

Given that for every $m$ the PDE yields a unique solution $u$, it is possible to state the following reduced problem:
$$
\min_{m} \tilde{J}(m) := J(u(m),m) \\
\text{subject to} \\ l_b \leq m \leq u_b\\
g(m) \leq 0
$$

Now the PDE-constraint is exactly satisfied at each optimisation iteration: therefore the optimisation loop can be terminated as soon as the functional is sufficiently reduced by the optimisation algorithm, without any feasibility iterations.

## Define the optimal control problem


In [ ]:
# Define and solve the variational problem above (dolfin adjoint exploits it several times for optimization; it is calles 'tape')

J = assemble((0.5 * inner(u - 0.5, u - 0.5)) * dx + f ** 2 * dx)

control = Control(m) # Single control. Change 'm' with the control variable chosen

control = [Control(m) for m in controls_list] # Multiple control. Change 'controls_list' with a list of control variables chosen

# N.B. In case of constant control parameters, use Constant() to define them

# N.B. To change the variational problem to deal with, clear the tape with 'set_working_tape(Tape())';
#      then solve the new variational problem (e.g. with a different parameter) and go ahead with optimization

# N.B. use 'ds' for integrals over boundaries
#      use 'dx(1)' or 'ds(1)' to integrate over the subdomain / boundary's portion marked with 1

# N.B. For vectors:  (grad(v))_ij = dv_i / dx_j
#                    (nabla_grad(v))_ij = dv_j / dx_i
#      For matrices: (div(v))_i = dv_ij / dx_j
#                    (nabla_div(v))_j = dv_ij / dx_i

In [ ]:
# Time-dependent functional J
# if J is time-dependent, all the needed forms must be assemble in the time-loop of the variational problem

u_n = project(Expression(("sin(2 * pi * x[0])", "cos(2 * pi * x[1])"), degree = 2),  V) # Starting control parameter
control = Control(u_n) # Control the initial condition for example

t = 0.0
timestep = Constant(0.01)
T = 0.1

Jtemp = assemble(inner(u_n, u_n) * dx)
Jlist = [Jtemp]
while (t <= T):
    t += float(timestep)
    solve(a == L, u, bc)
    u_n.assign(u)

    Jtemp = assemble(inner(u, u) * dx) 
    Jlist.append(Jtemp) # Jlist stores the integrals of <u(x,t),u(x,t)>*dx for each time step for example

# Example 1: # J as the integral of <u(x,T),u(x,T)>*dx
J = Jlist[-1] 
# Example 2: J as the ratio of two integrals at different time
J = Jlist[3] * Jlist[0]**(-1)
# Example 3: J as the double integral <u(x,t),u(x,t)>*dx*dt (trapezoidal rule used to approx it)
#            This could be also computed with 'Jtemp += assemble(inner(u, u) * dx)' in the loop
J = 0
for i in range(1, len(Jlist)):
    J += 0.5 * (Jlist[i-1] + Jlist[i]) * float(timestep)

In [ ]:
# Time-dependent control variable
# Create one control function for each timestep in the model and store all controls in a dictionary that maps timestep to control function

# Example: J = \int_0^T \int_{\Omega} y^2 * dx * dt + \int_0^T \int_{\Omega} (u')^2 * dx * dt
# First integral approx in time with trapezidal rule, the derivative in the second integral approx with incremental ratio

from collections import OrderedDict

dt = Constant(0.1)
T = 2

controls = OrderedDict()
t = float(dt)
while t <= T:
    controls[t] = Function(V) # Initialize one control for each time-step
    t += float(dt)

y = Function(V, name = "solution") # Initial condition
t = float(dt)
J = 0.5 * float(dt) * assemble((y)**2 * dx)

u = Function(V)

while t <= T:
    
    u.assign(controls[t]) # Update source term from control array
    
    f.t = t # Update time-dependent data
    
    solve(a == L, y) # Solve PDE
    
    # Implement a trapezoidal rule
    if t > T - float(dt):
        weight = 0.5
    else:
        weight = 1
    J += weight * float(dt) * assemble((y)**2 * dx)

    t += float(dt) # Update time

# Add regularisation term to J
alpha = Constant(1e-1)
regularisation = alpha / 2 * sum([1 / dt * (fb - fa)**2 * dx for fb, fa in zip(list(ctrls.values())[1:], list(ctrls.values())[:-1])])
J = J + assemble(regularisation)

## Compute derivatives

In [ ]:
dJ = compute_gradient(J, control)
dJ1, dJ2 = compute_gradient(J, [Control(u), Control(f)])

## Solve the optimisation problem


In [ ]:
# Solve the PDE-constrained optimisation problem without bounds or extra constraints

Jhat = ReducedFunctional(J, control)
Jhat = ReducedFunctional(J, [control1, control2, ...])

m_opt = minimize(Jhat) # L-BFGS-B is used by default
m_opt = minimize(Jhat, method = ...) # Run print_optimization_methods() to see the available methods
m_opt = minimize(Jhat, method = 'SLSQP', tol = 1e-10, options = {'maxiter' : 200,
                                                                 'gtol' : 1e-3, # Stop if the gradient norm drops below gtol
                                                                 'disp': True}) # Print convergence messages.
m_opt = maximize(Jhat) # All the command for 'minimize' can work also for 'maximize'

In [ ]:
# Methods and options available

print_optimization_methods()

from scipy.optimize import show_options
show_options('minimize', method = ...)
show_options('maximize', method = ...)

In [ ]:
# Solve the PDE-constrained optimisation problem with bounds for the control variable

m_opt = minimize(Jhat, bounds = (control_lb, control_ub)) # control_lb and control_ub are of the same type as the controller
m_opt = minimize(Jhat, bounds = (0.0, 1.0)) # Float case
m_opt = minimize(Jhat, bounds = [[control1_lb, control2_lb, ...], [control1_ub, control2_ub, ...])

In [ ]:
# Solve the PDE-constrained optimisation problem with inequality constraint g(m)>=0
# Example: g(m) = 0.4 - \int{m*dx} >= 0

Jhat = ReducedFunctional(J, control)

# method 1: write the constraint as \int{(0.4 / |\Omega| - m) * dx} and use 'UFLInequalityConstraint'
k = Constant(0.4)
domain_size = Constant(2)
MyConstraint = UFLInequalityConstraint((k / domain_size - m) * dx, m) # Change the first 'm' with the control variable

m_opt = minimize(Jhat, constraints = MyConstraint)

# method 2
class MyConstraint(InequalityConstraint): # Use 'EqualityConstraint' as base class in case of equality constraint
    
    def __init__(self, k):
        self.k = float(k)
        self.smass = assemble(TestFunction(V) * Constant(1) * dx) # Directly assemble the mass matrix to compute the integral
        self.tmpvec = Function(V)

    def function(self, m): # Method to compute the constraint value needed
        from pyadjoint.reduced_functional_numpy import set_local
        set_local(self.tmpvec, m) # Use local indices
        integral = self.smass.inner(self.tmpvec.vector())
        return [self.k - integral]

    def jacobian(self, m): # Method to compute the Jacobian needed
        return [-self.smass]

    def length(self): # Method to return the number of components in the constraint needed
        return 1
    
    def output_workspace(self):
        return [0.0]

m_opt = minimize(Jhat, constraints = MyConstraint(k))

In [ ]:
# Callbacks: function executed after every optimisation iteration to save or plot the functional or parameter values
# method 1: callbacks are executed whenever the functional is evaluated

def eval_cb(j, m):
    print "j = %f, m = %f." % (j, float(m))
    File("name.pvd") << m # Save solution whenever J is evaluated

def derivative_cb(j, dj, m):
    print "j = %f, dj = %f, m = %f." % (j, dj, float(m))

Jhat = ReducedFunctional(J, control, eval_cb = eval_cb, derivative_cb = derivative_cb)

# method 2
def iter_cb(m):
    print "m = ", m

Jhat = ReducedFunctional(J, control)
sol_opt = minimize(Jhat, callback = iter_cb)

In [ ]:
# Use an artificial neural network to model a function and train it

try:
    from ufl_dnn.neural_network import ANN
except ImportError:
    !pip install git+https://github.com/sebastkm/hybrid-fem-nn.git@master
    from ufl_dnn.neural_network import ANN

layers = [2, 10, 1] # 2 inputs, 1 hidden layer with 10 neurons and 1 output
bias = [True , True]
net = ANN(layers, bias = bias, mesh = mesh)

def f_ANN(u1, u2):
    return net([u1, u2])

# Solve the variational problem with f_ANN inside a first time
# Define the cost functional J

Jhat = ReducedFunctional(J, net.weights_ctrls())
opt_weights = minimize(Jhat, method ="L-BFGS-B", tol = 1e-6, options = {'disp': True, "maxiter":100})
net.set_weights(opt_weights)

# Solve the variational problem another time to compute the final results
# To plot f_ANN, evaluate it on a grid of inputs and plot the obtained values

In [ ]:
# Solve with Moola package

try:
    import moola
except ImportError:
    !pip install moola
    import moola

Jhat = ReducedFunctional(J, control)

problem = MoolaOptimizationProblem(Jhat)
control_moola = moola.DolfinPrimalVector(m) # Change 'm' with the control variable chosen

# Case with multiple control variables
control1_moola = moola.DolfinPrimalVector(m1)
control2_moola = moola.DolfinPrimalVector(m2)
control_moola = moola.DolfinPrimalVectorSet([control1_moola, control2_moola])

solver = moola.NewtonCG(problem, control_moola, options={'gtol': 1e-9,
                                                   'maxiter': 20,
                                                   'display': 3,
                                                   'ncg_hesstol': 0})

solver = moola.BFGS(problem, control_moola, options={'jtol': 0,
                                               'gtol': 1e-9,
                                               'Hinit': "default",
                                               'maxiter': 100,
                                               'mem_lim': 10})

sol = solver.solve()
sol_opt = sol['control'].data

In [ ]:
# Solve with IPOPT

try:
    from pyadjoint import ipopt
except ImportError:
    print("""This example depends on IPOPT and Python ipopt bindings. \
  When compiling IPOPT, make sure to link against HSL, as it \
  is a necessity for practical problems.""")
    raise

parameters["std_out_all_processes"] = False # Turn off redundant output in parallel

Jhat = ReducedFunctional(J, control)

problem = MinimizationProblem(Jhat, bounds = (l_b, u_b), constraints = MyConstraint(k)) 
parameters = {"acceptable_tol": 1.0e-3, "maximum_iterations": 100}
solver = IPOPTSolver(problem, parameters = parameters)
sol_opt = solver.solve()

# Postprocessing

In [ ]:
# Plot control variable
plot(sol_opt, title="sol_opt")

# Plot controlled solution
m.assign(sol_opt) # Change 'm' with the control variable chosen
solve(a == L, u, bc)

clear_output(wait=False)
plot(u)

In [ ]:
# Save solution
output_file = HDF5File(MPI.comm_world, "name.h5", "w") # There is also XDMFile() for files .xdmf
output_file.write(sol_opt, "solution")
output_file.close()

File('path/name.pvd') << sol_opt # Paraview Format

# Load solution
sol_opt_new = Function(V)
input_file = HDF5File(MPI.comm_world, "name.h5", "r")
input_file.read(sol_opt, "solution")
input_file.close()

## Compute errors

In [ ]:
sol_ex = Expression("1 / (1 + 4 * pow(pi, 4)) * sin(pi * x[0]) * sin(pi * x[1])", degree = 3)
u_ex = Expression("1 / (2 * pow(pi, 2)) * f", f = f_ex, degree = 3)

m.assign(sol_opt) # Change 'm' with the control variable chosen
solve(a == L, u, bc)
control_error = errornorm(sol_ex, sol_opt, 'L2')
state_error = errornorm(u_ex, u, 'L2')

print("h(min):           %e." % mesh.hmin())
print("Error in state:   %e." % state_error)
print("Error in control: %e." % control_error)

## Taylor test

In [ ]:
control = Control(m) # Change 'm' with the control variable chosen

# 'h' must be of the same type as the controller
# if the controller is a constant:
h = Constant(0.0001)
# if the controller is a function:
h = Function(V)
h.vector()[:] = 0.1

Jhat = ReducedFunctional(J, control)
conv_rate = taylor_test(Jhat, m, h)  # Change 'm' with the control variable chosen; if the gradient has been computed correctly, the convergence rates found should be 2
conv_rate = taylor_test(Jhat, m, h, dJdm = 0)  # Change 'm' with the control variable chosen; if the gradient has been computed correctly, the convergence rates found should be 1

# Taylor remainders are not correct if the model manually modifies function values or the entries of assembled matrices and vectors, if the model is not differentiable or if there is a bug in dolfin-adjoint

# RBniCS - Reduced Order Model x PDE

Consider a parameter-dependent unsteady PDE in the domain $\Omega\times[0,T]$:
<center>for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, find $u(\boldsymbol{\mu})$ such that</center>

$$
\begin{cases}
	G(y(t;\boldsymbol{\mu});\boldsymbol{\mu}) = 0 & \text{in } \Omega\times[0,T],\\
	\text{B.C.} & \text{on } \partial \Omega\times[0,T],\\
    \text{I.C.} & \text{on } \Omega\times\{t=0\}
\end{cases}
$$

where $y(t;\boldsymbol{\mu})$ is the state and $\boldsymbol{\mu}$ is a vector of parameters.

The corresponding weak formulation reads as follow:
<center>for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$,  for $t\in[0,T]$, find $y(t;\boldsymbol{\mu})\in\mathbb{V}$ such that</center>

$$m\left(\partial_t y(t;\boldsymbol{\mu}),v;\boldsymbol{\mu}\right) + a\left(y(t;\boldsymbol{\mu}),v;\boldsymbol{\mu}\right)+c\left(y(t;\boldsymbol{\mu}),v;\boldsymbol{\mu}\right)=f(v;\boldsymbol{\mu})\quad \forall v\in\mathbb{V},\quad \forall t\in[0,T]$$

where

* $\mathbb{V}$ is the function space for the state $y$;
* $m(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is the parametrized bilinear accounting for the term with the time derivative of y
* $a(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is the parametrized bilinear form;
* $c(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is the parametrized bilinear form accounting for the non-linearities;
* $f(\cdot; \boldsymbol{\mu}): \mathbb{V} \to \mathbb{R}$ is the parametrized linear form

Let $s(t;\boldsymbol{\mu})$  the output of interest computed starting from $y(t;\boldsymbol{\mu})$.

## Affine decomposition

In order to reduce the cost when assembling ROM matrices and vectors, the parameter-dependent part and the parameter-independent one should be separated; this is called affine decomposition. In formulas:

$$ m(y,v;\boldsymbol{\mu})=\sum_{i=1}^{N_m} \Theta^{m}_i(\boldsymbol{\mu}) m_i(y,v) $$
$$ a(y,v;\boldsymbol{\mu})=\sum_{i=1}^{N_a} \Theta^{a}_i(\boldsymbol{\mu}) a_i(y,v) $$
$$ c(y,v;\boldsymbol{\mu})=\sum_{i=1}^{N_c} \Theta^{c}_i(\boldsymbol{\mu}) c_i(y,v) $$
$$ f(v; \boldsymbol{\mu}) = \sum_{i=1}^{N_f} \Theta^{f}_i(\boldsymbol{\mu}) f_i(y,v) $$

If the problem is not affine, EIM or DEIM can retrieve this setting by an approximation procedure.

## Problem definition

In [ ]:
import rbnics.utils.config
assert "dolfin" in rbnics.utils.config.config.get("backends", "required backends")
from rbnics import *

# Extra imports for online stabilization for advection dominated problems
from problems import *
from reduction_methods import *

# Extra imports for uncertainty quantification
from problems import *
from reduction_methods import *
from sampling.distributions import *
from sampling.weights import *

In [ ]:
# Implement the numerical discretization of the problem in the following class
# It is possible to define a new problem "Type" if not present yet

@ExactParametrizedFunctions() # Decorator for exact treatment of non-affine terms or non-linear terms
@EIM() # Decorator for Empirical Interpolation Method
@DEIM() # Decorator for Discrete Empirical Interpolation Method
# We can also apply different methods offline and online; for example:
@ExactParametrizedFunctions("offline")
@DEIM("online", basis_generation = "Greedy")

@WeightedUncertaintyQuantification() # Decorator for uncertainty quantification

@OnlineStabilization() # Decorator for advection dominated problems

@SCM() # Decorator for Successive Constraint Method to estimate the stability factor for Greedy

@PullBackFormsToReferenceDomain() # Decorators for parameter-dependent domain
@ShapeParametrization(...) # e.g. ShapeParametrization(("x[0]", "x[1]"), # subdomain 1
                           #                           ("mu[0]*(x[0] - 1) + 1", "x[1]")) # subdomain 2
@AffineShapeParametrization(...) # Affine version of ShapeParametrization

class MyProblem(Type): # "Type" can be one of the following:
                       # EllipticCoerciveCompliantProblem, EllipticCoerciveProblem, NonlinearEllipticProblem
                       # ParabolicCoerciveProblem, NonlinearParabolicProblem,
                       # StokesProblem, StokesUnsteadyProblem, NavierStokesProblem, NavierStokesUnsteadyProblem (see the tutorial for the formulation)
                       # GeostrophicProblem (see the tutorial for the formulation)
                           
    # Default initialization of members (call standard initialization + store FEniCS data structures for assembly)
    @generate_function_space_for_stability_factor # Decorator to estimate stability factor for Greedy
    def __init__(self, V, **kwargs):
        # Call the standard initialization
        Type.__init__(self, V, **kwargs) # Change "Type"
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        self.y = TrialFunction(V)
        self.v = TestFunction(V)
        self.dx = Measure("dx")(subdomain_data = self.subdomains)
        self.ds = Measure("ds")(subdomain_data = self.boundaries)
        
        # Set hyperparameters and options if needed
        self.gamma = 1.0
        self.fun1 = Expression("sin(2*pi*x[0])*sin(2*pi*x[1])", element = self.V.ufl_element()) # Non-linear function
        self.fun2 = ParametrizedExpression(self, "exp(- 2 * pow(x[0] - mu[0], 2) - 2 * pow(x[1] - mu[1], 2))",
                                          mu = (0., 0.), element = V.ufl_element()) # Non-affine function: we cannot use self.mu here because it has not been initialized yet
        self._eigen_solver_parameters.update({"bounding_box_minimum": {"problem_type": "gen_hermitian", "spectral_transform": "shift-and-invert",
                                                                       "spectral_shift": 1.e-5, "linear_solver": "mumps"},
                                              "bounding_box_maximum": {"problem_type": "gen_hermitian", "spectral_transform": "shift-and-invert",
                                                                       "spectral_shift": 1.e5, "linear_solver": "mumps"},
                                              "stability_factor": {"problem_type": "gen_hermitian", "spectral_transform": "shift-and-invert",
                                                                   "spectral_shift": 1.e-5, "linear_solver": "mumps"}})
        self._nonlinear_solver_parameters.update({"linear_solver": "mumps", "maximum_iterations": 20, "report": True, "line_search": "wolfe"})
        self._time_stepping_parameters.update({"report": True,"snes_solver": {"linear_solver": "mumps", "maximum_iterations": 20, "report": True}})
        
    # Return custom problem name
    def name(self):
        return "MyProblem"

    # Return the stability factor for Greedy
    def get_stability_factor_lower_bound(self):
        return min(self.compute_theta("a"))

    
    # Return theta multiplicative terms of the affine expansion of the problem
    @compute_theta_for_stability_factor # Decorator to estimate the stability factor for Greedy
    @compute_theta_for_derivatives # Decorator for non-linear problems
    @compute_theta_for_supremizers # Decorator for supremizer operator to ensure inf-sup condition
    def compute_theta(self, term):
        mu = self.mu # Parameters mu accessed in this way, then use mu[i] to access the i-th parameter
        if term == "m":
            theta_m0 = ... # Fill and add the theta_mi terms
            ...
            return (theta_m0, ...)
        elif term == "a":
            theta_a0 = ... # Fill and add the theta_ai terms
            ...
            return (theta_a0, ...)
        elif term == "c": 
            theta_c0 = ... # Fill and add the theta_ci terms
            ...
            return (theta_c0, ...)
        elif term == "f":
            theta_f0 = ... # Fill and add the theta_fi terms
            ...
            return (theta_f0, ...)
        elif term == "s":
            theta_s0 = ... # Fill and add the theta_si terms
            ...
            return (theta_s0, ...)
        elif term == "dirichlet_bc":
            theta_bc0 = ... # Fill and add the theta_bci terms
            ...
            return (theta_bc0, ...)
        else:
            raise ValueError("Invalid term for compute_theta().")
    # N.B. If only theta_a0 is present, return (theta_a0,) instead of returning (theta_a0). The same for the other cases.
    # N.B. If the state is a vector (e.g. y = [u,p]), then consider 'dirichlet_bc_u' and 'dirichlet_bc_p'
    
    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    @assemble_operator_for_stability_factor # Decorator to estimate the stability factor for Greedy
    @assemble_operator_for_derivatives # Decorator for non-linear problems
    @assemble_operator_for_supremizers # Decorator for supremizer operator to ensure inf-sup condition
    def assemble_operator(self, term):
        dx = self.dx
        ds = self.ds
        y = self.y
        v = self.v
        if term == "m":
            m0 = ... # Fill and add the mi terms. N.B. here y stands for dy/dt; e.g. m0 = y * v * dx stands for dy/dt * v * dx
            ....
            return (m0, )
        elif term == "a":     
            a0 = ... # Fill and add the ai terms; e.g. a0 = inner(grad(y), grad(v)) * dx(1)
            ...
            return (a0, ...)
        elif term == "c":
            c0 = ... # Fill and add the ci terms; e.g. c0 = (exp(mu[1] * u) - 1) / mu[1] * v * dx
            ...
            return (c0, ...)
        elif term == "f":
            f0 = ... # Fill and add the fi terms; e.g. f0 = v * ds(1)
            ...
            return (f0, ...)
        elif term == "s":
            s0 = v * dx # Fill and add the si terms; e.g. s0 = v * dx
            ...
            return (s0, ...)
        elif term == "dirichlet_bc":
            bc0 = ... # Fill and add the bci terms; e.g. [DirichletBC(self.V, Constant(0.0), self.boundaries, 3), DirichletBC(self.V, Constant(1.0), self.boundaries, 2)]
            ...
            return (bc0, ...)
        elif term == "inner_product": # Inner product used in the space V
            x0 = ... # e.g. x0 = inner(grad(y), grad(v)) * dx
            return (x0,)
        elif term == "projection_inner_product":
            x0 = ...
            return (x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")
        # N.B. If only a0 is present, return (a0,) instead of returning (a0). The same for the other cases.
        # N.B. If the state is a vector (e.g. y = [u,p]), then consider 'dirichlet_bc_u', 'dirichlet_bc_p', 'inner_product_u', 'inner_product_p'

    # Auxiliary function useful inside the class
    def aux(self, ...):
        ...
        return ...

In [ ]:
# Define or import the mesh, the subdomains and the boundaries

In [ ]:
# Define the functional space V

In [ ]:
# Define the problem

problem = MyProblem(V, subdomains = subdomains, boundaries = boundaries)
mu_range = [(,),...] # Set the range for the param; e.g. mu_range = [(0.1, 10.0), (-1.0, 1.0)]
problem.set_mu_range(mu_range)

# Parameter for unsteady problems
dt = ...
T = ...
problem.set_time_step_size(dt)
problem.set_final_time(T)

## Reduced basis methods

In [ ]:
# Choose a Reduce basis method

reduction_method = ReducedBasis(problem) # Greedy algorithm
reduction_method = PODGalerkin(problem) # POD-Galerkin

In [ ]:
# Set parameters for the Reduce basis method

Nmax = ...  # Set max dimension of the reduced space
toll = ...  # Set tolerance

reduction_method.set_Nmax(Nmax) # Possible extra input: POD_Greedy, nested_POD, EIM, DEIM, SCM
reduction_method.set_tolerance(toll) # Possible extra input: POD_Greedy, nested_POD, EIM, DEIM, SCM

In [ ]:
# Offline phase

N_train = ... # Set size of the training set
reduction_method.initialize_training_set(N_train) # Possible extra input: EIM, DEIM, SCM
reduction_method.initialize_training_set(N_train, sampling = ..., weight = ...)
# Examples: sampling = LogUniformDistribution()
#           sampling = LinearlyDependentUniformDistribution()
#           sampling = BetaDistribution(beta_a, beta_b), weight = BetaWeight(beta_a, beta_b) where beta_a and beta_b are vectors of length nparam (nparam=len(mu))
reduced_problem = reduction_method.offline()

In [ ]:
# Online phase

online_mu = ... # Choose the parameters to use online; e.g. online_mu = (3.5, 0.02, 11.0)

reduced_problem.set_mu(online_mu)
reduced_solution = reduced_problem.solve() # Possible extra input: EIM, DEIM, online_stabilization = True/False

In [ ]:
reduced_problem.export_solution(filename = "online_solution")
reduced_problem.export_error(filename = "online_error")

In [ ]:
reduced_problem.compute_output()

In [ ]:
plot(reduced_solution, reduced_problem = reduced_problem)
plot(reduced_solution, reduced_problem = reduced_problem, component = ...) # Plot single components in case of vectorial state 
plot(reduced_solution, reduced_problem = reduced_problem, every = 5, interval = 500) # Plot for unsteady problems

In [ ]:
basis_functions = reduced_problem.basis_functions
plot_phase_space(reduced_problem, reduced_solution, basis_functions, 0.0) # Plot for unsteady problems

In [ ]:
# Error and speed-up analysis

N_test = ... # Set size of the test set

reduction_method.initialize_testing_set(N_test) # Possible extra input: EIM, DEIM, SCM
reduction_method.initialize_testing_set(N_test, sampling = ...) # Uncertainty quantification case
reduction_method.error_analysis()
reduction_method.error_analysis(filename="error_analysis")
reduction_method.error_analysis(filename="error_analysis", with_respect_to = exact_problem) # Possible extra input: EIM, DEIM
reduction_method.speedup_analysis(filename="speedup_analysis")

# RBniCS - Reduced Order Model x OCP

The optimal control problem reads as follows: <center> $\underset{y,u}{min} \; J(y(\boldsymbol{\mu}), u(\boldsymbol{\mu});\boldsymbol{\mu})$ <center>

subject to
    
$$\begin{cases}
	G(y,u;\boldsymbol{\mu}) = 0 & \text{in } \Omega,\\
	\text{B.C.} & \text{on } \partial \Omega,\\
\end{cases}$$

where the constraint is a parameter-dependent PDE in the domain $\Omega$ with parameters $\boldsymbol{\mu}$, state $y(\boldsymbol{\mu})$ and control variable $u(\boldsymbol{\mu})$;
    
Let $h(\boldsymbol{\mu})$  the output of interest computed starting from $y(\boldsymbol{\mu})$.
    
Using the Lagrangian functional, the problem becomes: <center> find $(y,p,u) \in \mathbb{Y} \times \mathbb{Q} \times \mathbb{U}\; : \; \nabla \mathcal{L}(y,p,u)[(z,q,v)]=0 \quad \forall (z,q,v) \in \mathbb{Y} \times \mathbb{Q} \times \mathbb{U}$ </center>

which gives:
<center>
    $
    \begin{cases}
        \mathcal{L}_{p} = f(q) + c(u,q) - a(y,q) \\
        \mathcal{L}_{y} = m(y,z) - g(y_d,z) - a^*(z,p) \\
        \mathcal{L}_{u} = n(u,v) + c^*(v,p)
    \end{cases}
    $
</center>

where
* $a(\cdot,\cdot; \boldsymbol{\mu}): \mathbb{Y} \times \mathbb{Q} \rightarrow \mathbb{R}$ is the parametrized bilinear form defined on the state $\times$ adjoint space;
* $c(\cdot,\cdot; \boldsymbol{\mu}): \mathbb{U} \times \mathbb{Q} \rightarrow \mathbb{R}$ is the parametrized bilinear form defined on the control $\times$ adjoint space;
* $f(\cdot; \boldsymbol{\mu}): \mathbb{Y} \rightarrow \mathbb{R}$ is the parametrized linear form defined on the state space;
* $m(\cdot,\cdot; \boldsymbol{\mu}): \mathbb{Z} \times \mathbb{Z} \rightarrow \mathbb{R}$ is the parametrized bilinear form defined on the state space;
* $n(\cdot,\cdot; \boldsymbol{\mu}): \mathbb{U} \times \mathbb{U} \rightarrow \mathbb{R}$ is the parametrized bilinear form defined on control space;
* $g(\cdot,\cdot; \boldsymbol{\mu}): \mathbb{Z} \times \mathbb{Y} \rightarrow \mathbb{R}$ is the parametrized bilinear form defined on state space;
* the functional space $\mathbb{Y}$ is the state space;
* the functional space $\mathbb{U}$ is the control space;
* the functional space $\mathbb{Q}$ is the adjoint space (often $\mathbb{Q}=\mathbb{Y}$)
* the functional space $\mathbb{Z} \supset \mathbb{Y}$


In [ ]:
import rbnics.utils.config
assert "dolfin" in rbnics.utils.config.config.get("backends", "required backends")
from rbnics import *

In [ ]:
# Implement the numerical discretization of the problem in the following class 
# It is possible to define a new problem "Type" if not present yet

@ExactParametrizedFunctions() # Decorator for exact treatment of non-affine terms or non-linear terms
@EIM() # Decorator for Empirical Interpolation Method
@DEIM() # Decorator for Discrete Empirical Interpolation Method
# We can also apply different methods offline and online; for example:
@ExactParametrizedFunctions("offline")
@DEIM("online", basis_generation = "Greedy")

@WeightedUncertaintyQuantification() # Decorator for uncertainty quantification

@OnlineStabilization() # Decorator for advection dominated problems

@SCM() # Decorator for Successive Constraint Method to estimate the stability factor for Greedy

@PullBackFormsToReferenceDomain() # Decorators for parameter-dependent domain
@ShapeParametrization(...) # e.g. ShapeParametrization(("x[0]", "x[1]"), # subdomain 1
                           #                           ("mu[0]*(x[0] - 1) + 1", "x[1]")) # subdomain 2
@AffineShapeParametrization(...) # Affine version of ShapeParametrization

class MyProblem(Type): # "Type" can be one of the following:
                       # EllipticOptimalControlProblem,
                       # StokesOptimalControlProblem (see the tutorial for the formulation)
                       # GeostrophicOptimalControlProblem (see the tutorial for the formulation)            
                           
    # Default initialization of members (call standard initialization + store FEniCS data structures for assembly)
    @generate_function_space_for_stability_factor # Decorator to estimate stability factor for Greedy
    def __init__(self, V, **kwargs):
        # Call the standard initialization
        Type.__init__(self, V, **kwargs) # Change "Type"
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        yup = TrialFunction(V)
        (self.y, self.u, self.p) = split(yup)
        zvq = TestFunction(V)
        (self.z, self.v, self.q) = split(zvq)
        self.dx = Measure("dx")(subdomain_data = self.subdomains)
        self.ds = Measure("ds")(subdomain_data = self.boundaries)
        
        # Set hyperparameters and options if needed
        self.gamma = 1.0
        self.fun1 = Expression("sin(2*pi*x[0])*sin(2*pi*x[1])", element = self.V.ufl_element()) # Non-linear function
        self.fun2 = ParametrizedExpression(self, "exp(- 2 * pow(x[0] - mu[0], 2) - 2 * pow(x[1] - mu[1], 2))",
                                          mu = (0., 0.), element = V.ufl_element()) # Non-affine function: we cannot use self.mu here because it has not been initialized yet
        self._eigen_solver_parameters.update({"bounding_box_minimum": {"problem_type": "gen_hermitian", "spectral_transform": "shift-and-invert",
                                                                       "spectral_shift": 1.e-5, "linear_solver": "mumps"},
                                              "bounding_box_maximum": {"problem_type": "gen_hermitian", "spectral_transform": "shift-and-invert",
                                                                       "spectral_shift": 1.e5, "linear_solver": "mumps"},
                                              "stability_factor": {"problem_type": "gen_hermitian", "spectral_transform": "shift-and-invert",
                                                                   "spectral_shift": 1.e-5, "linear_solver": "mumps"}})
        self._nonlinear_solver_parameters.update({"linear_solver": "mumps", "maximum_iterations": 20, "report": True, "line_search": "wolfe"})
        self._time_stepping_parameters.update({"report": True,"snes_solver": {"linear_solver": "mumps", "maximum_iterations": 20, "report": True}})
        
    # Return custom problem name
    def name(self):
        return "MyProblem"

    # Return the stability factor for Greedy
    def get_stability_factor_lower_bound(self):
        return min(self.compute_theta("a"))
    
    # Return theta multiplicative terms of the affine expansion of the problem
    @compute_theta_for_stability_factor # Decorator to estimate the stability factor for Greedy
    @compute_theta_for_derivatives # Decorator for non-linear problems
    @compute_theta_for_supremizers # Decorator for supremizer operator to ensure inf-sup condition
    def compute_theta(self, term):
        mu = self.mu # Parameters mu accessed in this way, then use mu[i] to access the i-th parameter
        if term in ("a", "a*"):
            theta_a0 = ... # Fill and add the theta_ai terms
            ...
            return (theta_a0, ...)
        elif term in ("c", "c*"): 
            theta_c0 = ... # Fill and add the theta_ci terms
            ...
            return (theta_c0, ...)
        elif term == "m":
            theta_m0 = ... # Fill and add the theta_mi terms
            ...
            return (theta_m0, ...)
        elif term == "n":
            theta_n0 = ... # Fill and add the theta_ni terms
            ...
            return (theta_n0, ...)
        elif term == "f":
            theta_f0 = ... # Fill and add the theta_fi terms
            ...
            return (theta_f0, ...)
        elif term == "g":
            theta_g0 = ... # Fill and add the theta_gi terms
            ...
            return (theta_g0, ...)
        elif term == "h": 
            theta_h0 = ... # Fill and add the theta_hi terms
            ...
            return (theta_h0, ...)
        elif term == "dirichlet_bc_y":
            theta_bc0 = ... # Fill and add the theta_bci terms
            ...
            return (theta_bc0, ...)
        else:
            raise ValueError("Invalid term for compute_theta().")
    # N.B. If only theta_a0 is present, return (theta_a0,) instead of returning (theta_a0). The same for the other cases.
    # N.B. If the state is a vector (e.g. y = [u,p]), then consider 'dirichlet_bc_u' and 'dirichlet_bc_p'           
            
    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    @assemble_operator_for_stability_factor # Decorator to estimate the stability factor for Greedy
    @assemble_operator_for_derivatives # Decorator for non-linear problems
    @assemble_operator_for_supremizers # Decorator for supremizer operator to ensure inf-sup condition
    def assemble_operator(self, term):
        dx = self.dx
        ds = self.ds
        y = self.y
        z = self.z
        p = self.p
        q = self.q
        u = self.u
        v = self.v        
        if term == "a":     
            a0 = ... # Fill and add the ai terms; e.g. a0 = inner(grad(y), grad(q)) * dx
            ...
            return (a0, ...)
        elif term == "a*":
            as0 = ... # Fill and add the asi terms; e.g. as0 = inner(grad(z), grad(p)) * dx
            ...
            return (as0, ...)
        elif term == "c":
            c0 = ... # Fill and add the ci terms; e.g. c0 = u * q * dx
            ...
            return (c0, ...)
        elif term == "c*":
            cs0 = ... # Fill and add the csi terms; e.g. cs0 = v * p * dx
            ...
            return (cs0, ...)
        elif term == "m":
            m0 = ... # Fill and add the mi terms; e.g. m0 = y * z * dx
            ...
            return (m0, ...)
        elif term == "n":
            n0 = ... # Fill and add the ni terms; e.g. n0 = u * v * dx
            ...
            return (n0, ...)
        elif term == "f":
            f0 = ... # Fill and add the fi terms; e.g. f0 = Constant(0.0) * q * dx
            ...
            return (f0, ...)
        elif term == "g":
            g0 = ... # Fill and add the gi terms; e.g. g0 = y_d * z * dx(1)
            ...
            return (g0, ...)
        elif term == "h":
            h0 = ... # Fill and add the hi terms;
            ...
            return (h0, ...)
         elif term == "dirichlet_bc_y":
            bc0 = ... # Fill and add the bci terms; e.g. bc0 = [DirichletBC(self.V.sub(0), Constant(1.0), self.boundaries, i) for i in range(1, 9)]
            ...
            return (bc0, ...)
        elif term == "dirichlet_bc_p":
            bc0 = ... #Fill and add the bci terms; e.g. bc0 = [DirichletBC(self.V.sub(2), Constant(0.0), self.boundaries, i) for i in range(1, 9)]
            ...            
            return (bc0, ...)
        elif term == "inner_product_y":
            x0 = ... # Fill and add the xi terms; e.g. x0 = inner(grad(y), grad(z)) * dx
            return (x0,)
        elif term == "inner_product_u":
            x0 = ... # Fill and add the xi terms; e.g. x0 = u * v * dx
            return (x0,)
        elif term == "inner_product_p":
            x0 = ... #Fill and add the xi terms; e.g. x0 = inner(grad(p), grad(q)) * dx
            return (x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")
        # N.B. If only a0 is present, return (a0,) instead of returning (a0). The same for the other cases.

    # Auxiliary function useful inside the class
    def aux(self, ...):
        ...
        return ...

In [ ]:
# Define or import the mesh, the subdomains and the boundaries

In [ ]:
# Define the functional space V for state, control and adjoint
scalar_element = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
element = MixedElement(scalar_element, scalar_element, scalar_element)
V = FunctionSpace(mesh, element, components=["y", "u", "p"])

In [ ]:
# Define the problem

problem = MyProblem(V, subdomains = subdomains, boundaries = boundaries)
mu_range = [(,),...] # Set the range for the param; e.g. mu_range = [(0.1, 10.0), (-1.0, 1.0)]
problem.set_mu_range(mu_range)

# Parameter for unsteady problems
dt = ...
T = ...
problem.set_time_step_size(dt)
problem.set_final_time(T)

## Reduced basis methods

In [ ]:
# Choose a Reduce basis method

reduction_method = ReducedBasis(problem) # Greedy algorithm
reduction_method = PODGalerkin(problem) # POD-Galerkin

In [ ]:
# Set parameters for the Reduce basis method

Nmax = ...  # Set max dimension of the reduced space
toll = ...  # Set tolerance

reduction_method.set_Nmax(Nmax) # Possible extra input: POD_Greedy, nested_POD, EIM, DEIM, SCM
reduction_method.set_tolerance(toll) # Possible extra input: POD_Greedy, nested_POD, EIM, DEIM, SCM

In [ ]:
# Offline phase

N_train = ... # Set size of the training set
reduction_method.initialize_training_set(N_train) # Possible extra input: EIM, DEIM, SCM
reduction_method.initialize_training_set(N_train, sampling = ..., weight = ...)
# Examples: sampling = LogUniformDistribution()
#           sampling = LinearlyDependentUniformDistribution()
#           sampling = BetaDistribution(beta_a, beta_b), weight = BetaWeight(beta_a, beta_b) where beta_a and beta_b are vectors of length nparam (nparam=len(mu))
reduced_problem = reduction_method.offline()

In [ ]:
# Online phase

online_mu = ... # Choose the parameters to use online; e.g. online_mu = (3.5, 0.02, 11.0)

reduced_problem.set_mu(online_mu)
reduced_solution = reduced_problem.solve() # Possible extra input: EIM, DEIM, online_stabilization = True/False

In [ ]:
reduced_problem.export_solution(filename = "online_solution")
reduced_problem.export_error(filename = "online_error")

In [ ]:
reduced_problem.compute_output()

In [ ]:
plot(reduced_solution, reduced_problem = reduced_problem, component = "y")
plot(reduced_solution, reduced_problem = reduced_problem, component = "u")
plot(reduced_solution, reduced_problem = reduced_problem, component = "p")
plot(reduced_solution, reduced_problem = reduced_problem, component = "...", every = 5, interval = 500) # Plot for unsteady problems

In [ ]:
basis_functions = reduced_problem.basis_functions
plot_phase_space(reduced_problem, reduced_solution, basis_functions, 0.0) # Plot for unsteady problems

In [ ]:
# Error and speed-up analysis

N_test = ... # Set size of the test set

reduction_method.initialize_testing_set(N_test) # Possible extra input: EIM, DEIM, SCM
reduction_method.initialize_testing_set(N_test, sampling = ...) # Uncertainty quantification case
reduction_method.error_analysis()
reduction_method.error_analysis(filename="error_analysis")
reduction_method.error_analysis(filename="error_analysis", with_respect_to = exact_problem) # Possible extra input: EIM, DEIM
reduction_method.speedup_analysis(filename="speedup_analysis")